### Create a dataframe on raw_metrics table

In [ ]:
df = sqlContext.read\
   .format("org.apache.spark.sql.cassandra")\
   .options(keyspace = "metrics", table = "raw_metrics")\
   .load()	

### Explain the query plan and view some data

In [ ]:
df.printSchema()

In [ ]:
df.explain()

In [ ]:
df.show()

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
import datetime
from pyspark.sql import functions as func
from pyspark.sql.functions import col
from pyspark.sql.functions import lit

### set time variables for date filtering

In [ ]:
time = datetime.datetime.now()
epochtime = int(time.strftime("%s"))
start_time = epochtime - 86400
compare_time = datetime.datetime.fromtimestamp(start_time)

### create a dataframe from the raw metrics

In [ ]:
rawmetrics = sqlContext.read.format("org.apache.spark.sql.cassandra")\
    .options(table="raw_metrics", keyspace="metrics").load()

### filter metrics to those in last 24 hours

In [ ]:
last_day = rawmetrics.where(rawmetrics.metric_time > compare_time)
last_day.show()

### aggregates

In [ ]:
averages = last_day.groupby('device_id').agg(func.avg('metric_value').alias('metric_avg'))
maximums = last_day.groupby('device_id').agg(func.max('metric_value').alias('metric_max'))
minimums = last_day.groupby('device_id').agg(func.min('metric_value').alias('metric_min'))

### rename id columns for uniqueness

In [ ]:
averages_a = averages.withColumnRenamed("device_id", "id")
maximums_a = maximums.withColumnRenamed("device_id", "maxid")
minimums_a = minimums.withColumnRenamed("device_id", "minid")

### join the tables above

In [ ]:
temp = averages_a.join(maximums_a, averages_a.id == maximums_a.maxid)
aggs = temp.join(minimums, temp.id == minimums.device_id).select('id','metric_min','metric_max','metric_avg')

### add columns to format for cassandra

In [ ]:
addday = aggs.withColumn("metric_day", lit(time))
addname = addday.withColumn("metric_name",lit("KWH"))
inserts = addname.withColumnRenamed("id","device_id")

### Save to a new table

In [ ]:
inserts.write.format("org.apache.spark.sql.cassandra").\
    options(table="daily_rollups", keyspace = "metrics").save(mode ="append")